In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("spotify_millsongdata.csv")

In [ ]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Cher,I Hate To Sleep Alone,You were the only thing I had \r\nHow could a...
1,Black Sabbath,Iron Man,I am iron man \r\nHas he lost his mind? \r\n...
2,Waterboys,When Will We Be Married,When will we be married Molly \r\nWhen will w...
3,Dave Matthews Band,If I Had A Boat,If I had a boat \r\nI'd go out on the ocean ...
4,Deep Purple,Money Talks,I was young and healthy \r\nExtremely wealthy...
5,David Allan Coe,Honey Don't,By David Allan Coe \r\nI'VE DONE SOME RAMBLIN...
6,Widespread Panic,Send Your Mind,While you're out there on the highway \r\nWhi...
7,Manowar,Brothers Of Metal,Strike while the iron is hot \r\nSteel is str...
8,Otis Redding,You Don't Miss Your Water,"In the beginning \r\nYou really loved me, oh ..."
9,Judy Garland,If Love Were All,I believe in doing what I can \r\nIn crying w...


In [ ]:
df.tail(5)

,artist,song,text
4995,The Killers,Why Do I Keep Counting?,"There's a plane, and I am flying \r\nThere's ..."
4996,Taylor Swift,Love Story,We were both young when I first saw you \r\nI...
4997,Europe,America,You'd think by now I've learned to fake it \r...
4998,Dire Straits,Angel Of Mercy,Well there's a Peter Pan moon shepherd's delig...
4999,Used,Burning Down The House,"Watch out, \r\nYou might get what you're afte..."


In [ ]:
df['text'][0]

"You were the only thing I had  \r\nHow could a good love go so bad  \r\nSomehow I guess I should have known  \r\nThat you would leave me here alone  \r\n  \r\nThey say you found somebody new  \r\nBut that won't change my love for you  \r\nHow could you leave me on my own  \r\nYou know I hate to sleep alone  \r\n  \r\nRemember how it used to be  \r\nWhen you were here with me  \r\nYou were the only one I've known  \r\nAnd how I hate to sleep alone  \r\n  \r\nSomeday, some way, somebody new  \r\nWill make you feel the way that I do  \r\nBut I just wish that you'd come home  \r\nYou know that I hate to sleep alone  \r\n  \r\nBut I just wish that you'd come home  \r\nYou know that I hate to sleep alone\r\n\r\n"

In [ ]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', '').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/jayapriya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a =[stemmer.stem(w) for w in token]
    return " ".join(a)

In [ ]:
token("you are beautiful")

'you are beauti'

In [ ]:
df['text'].apply(lambda x: token(x))

0       you were the onli thing i had how could a good...
1       i am iron man ha he lost hi mind ? can he see ...
2       when will we be marri molli when will we be we...
3       if i had a boat i 'd go out on the ocean and i...
4       i wa young and healthi extrem wealthi i swung ...
                              ...                        
4995    there 's a plane , and i am fli there 's a mou...
4996    we were both young when i first saw you i clos...
4997    you 'd think by now i 've learn to fake it you...
4998    well there 's a peter pan moon shepherd 's del...
4999    watch out , you might get what you 're after ....
Name: text, Length: 5000, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
matrix = tfid.fit_transform(df['text'])

In [ ]:
similer = cosine_similarity(matrix)

In [ ]:
similer[1]

array([0.01221373, 1.        , 0.01057162, ..., 0.00563479, 0.0214025 ,
       0.00746042])

In [ ]:
df[df['song']=='Angel Of Mercy'].index[0]

4998

Recommender Function

In [ ]:
def recommender(song_name):
    idx = df[df['song']== song_name].index[0]
    distance = sorted(list(enumerate(similer[idx])), reverse=True, key = lambda x:x[1])
    song = []
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [ ]:
recommender("Angel Of Mercy")

['Angel Down',
 'Angel Of The Morning',
 'Angel',
 'Little Angel',
 'Earth Boy',
 "Sinner's Prayer",
 'Devil In Disguise',
 'Have Mercy',
 'Rest',
 "A Little Heaven's Rubbing Off On Me",
 'Mercy',
 'Angels Flying',
 "Ali D'oro",
 'Lullabye',
 "Angel's Lullaby",
 "Don't Look Back",
 'Shy Violet',
 'Tonight Tonight',
 'Angel Dust',
 'Like An Angel Passing Through My Room']

In [ ]:
import pickle

In [ ]:
pickle.dump(similer, open("similarity", "wb"))

In [ ]:
pickle.dump(df, open("df", "wb"))